In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install autocorrect


In [ ]:
pip install emoji

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from textblob import TextBlob as tb
from autocorrect import Speller as sp
import emoji
import nltk
from nltk.tokenize import word_tokenize as wt,sent_tokenize as st
from nltk.corpus import stopwords
from nltk import PorterStemmer,LancasterStemmer,SnowballStemmer
from nltk import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

**1.Problem Statement**



* Create a model to classify the Drug type based on their features



**2. Data Collection**

In [ ]:
# collect the dataset from kaggle,which is used to create a good model for classification
data = pd.read_csv(r"/content/drive/MyDrive/ML Tasks/Project 3 and assignment 8 to 10/drug200 (1).csv")
data.head()

**3. EDA and Pre-processing**

In [ ]:
data.shape   # data set contains 200 rows and 6 columns

In [ ]:
data.info()

This data contains some numerical data and categorical data.

In [ ]:
data.duplicated().sum()    # there is no duplicated values are here.

In [ ]:
data.Drug.value_counts()  # im balanced data

In [ ]:
data.Sex.unique()     # Sex feature contains two values male as M and female as F

In [ ]:
data.BP.unique()      # BP feature contains three values HIGH,LOW and NORMAL.

In [ ]:
data.Cholesterol.unique()   # Cholesterol feature contains two values HIGH and NORMAL

In [ ]:
data.Drug.unique()         # Drug feature contains FIVE values DrugY,drugC,drugX,drugA and drugB.

In [ ]:
sns.countplot(data.Drug)
plt.show()

In [ ]:
fv = data.iloc[:,:-1]    # split data into features and class variables
cv = data.iloc[:,-1]

In [ ]:
fv.head()

In [ ]:
cv.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
lbe = LabelEncoder()    # create an instance of Lable encoder class

In [ ]:
# convert categorical data into numerical using labelencoder
fv["Sex"] = lbe.fit_transform(fv["Sex"])
fv["BP"] = lbe.fit_transform(fv["BP"])
fv["Cholesterol"] = lbe.fit_transform(fv["Cholesterol"])
cv = lbe.fit_transform(cv)



In [ ]:
fv.head()

In [ ]:
cv = pd.Series(cv)
cv.head()

**5. EDA**

In [ ]:
# find the relation between features
# Correlation matrix
corr_matrix = fv.corr()
corr_matrix

In [ ]:
sns.heatmap(corr_matrix, annot=True, cmap='YlGnBu')
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# split the features and class variable into x_train,y_train,x_test,y_test
x_train,x_test,y_train,y_test = train_test_split(fv,cv,test_size=0.2,random_state=3,stratify = cv)

In [ ]:
x_train.head()

Based on above graph and table, all the features are indipendent only.So we will go with Naive Bayes theroem

**Q-Q plot**

*   Sorting values each feature
*   find 100 percentiles



In [ ]:
import scipy.stats as ss

In [ ]:
ss.probplot(x_train["Age"],dist = "norm",fit = True,plot = plt)   # Age feature doesn't follow gaussian distribution
plt.show()

In [ ]:
ss.probplot(x_train["Na_to_K"],dist = "norm",fit = True,plot = plt)    # Na_to_K features doesn't follow gaussian distribution.
plt.show()


so if we perform Guassian distribution , all the numerical variables are follows gaussian only.So we will transformation to convert gaussian distribution

**6.Feature Engineering**

In [ ]:
from sklearn.preprocessing import FunctionTransformer,PowerTransformer

In [ ]:
ft = FunctionTransformer(func = np.log)
ss.probplot(ft.transform(x_train.Na_to_K),dist = "norm",fit = True,plot = plt)
plt.show()

# this is doesn't follows Gaussian perfectly, so we will use another functional transformation

In [ ]:
ft = FunctionTransformer(func = np.square)
ss.probplot(ft.transform(x_train.Na_to_K),dist = "norm",fit = True,plot = plt)
plt.show()

# this is doesn't follows Gaussian perfectly, so we will use another functional transformation.It is very bad .

In [ ]:
ft = FunctionTransformer(func = np.sqrt)
ss.probplot(ft.transform(x_train.Na_to_K),dist = "norm",fit = True,plot = plt)
plt.show()

# this is doesn't follows Gaussian perfectly, so we will use another functional transformation

In [ ]:
def reci(x):
    return 1/x
ft = FunctionTransformer(func = reci)
ss.probplot(ft.transform(x_train.Na_to_K),dist = "norm",fit = True,plot = plt)
plt.show()

# this is doesn't follows Gaussian perfectly, so we will use another functional transformation

In [ ]:
pt = PowerTransformer()
ss.probplot(pt.fit_transform(x_train.Na_to_K.values.reshape(-1, 1)).ravel(),dist = "norm",fit = True,plot = plt)
plt.show()

 # this is follows Gaussian distribution, so we will go with Power transformation (yeo-johnson)

In [ ]:
x_train["Na_to_K"] = pt.fit_transform(x_train.Na_to_K.values.reshape(-1, 1)).ravel()     # Transfer x_train["Na_to_K"] to Gaussian

In [ ]:
x_train

Based on above graphs, all the features follows Gaussian distribution,So we performe Gaussian Naive bayes

**7.Training**

In [ ]:
pip install mixed-naive-bayes

In [ ]:
from mixed_naive_bayes import MixedNB

In [ ]:
# create an object of MixedNB class
mb = MixedNB(categorical_features = [1,2,3])   # it will perform both categorical and numerical Naive bayes.

# fit x_train and y_train based on Mixed NB
model = mb.fit(x_train,y_train)

then finally we create a model for the Mixed Naive bayes

**finding precission score of model**

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score,recall_score,RocCurveDisplay,f1_score

confusion_matrix(y_test,model.predict(x_test))

In [ ]:
precision_score(y_test,model.predict(x_test),average="weighted")     # it will give 95%

In [ ]:
precision_score(y_test,model.predict(x_test),average="micro")

In [ ]:
precision_score(y_test,model.predict(x_test),average="macro")     # it will give 98% using precision_score

In [ ]:
recall_score(y_test,model.predict(x_test),average="weighted")

In [ ]:
recall_score(y_test,model.predict(x_test),average="micro")       # it will give 95% using recall_score

In [ ]:
recall_score(y_test,model.predict(x_test),average="macro")

In [ ]:
f1_score(y_test,model.predict(x_test),average="weighted")

In [ ]:
f1_score(y_test,model.predict(x_test),average="micro")

In [ ]:
f1_score(y_test,model.predict(x_test),average="macro")

It will give the accuracy rate of 98% by using Precision_score

**8.Testing**

In [ ]:
lbe.classes_    # class labels of Drug column sequencially.

In [ ]:
dict = {}
for i in lbe.classes_:                        # create a dictionary index of lbe.classes_ as key and lbe.classes_ as value
  dict[list(lbe.classes_).index(i)] = i
dict

In [ ]:
m = x_test.iloc[[2]]
pred = model.predict(m)   # it is used to shows that predicted value
print(dict[pred[0]])      # it will give Class label of drugs based on features

i will give a query point from x_test.iloc[[2]] as input to the model,then it will returns class label is drugA